In [3]:
import csv 
import pandas as pd
import numpy as np
import os
import plotly  as py
import plotly.express as px
import plotly.graph_objs as go
folder = "C:/Users/user/Documents/GitHub/Martin Shine Blog Posts/"
data = "C:/Users/user/Documents/GitHub/Martin Shine Blog Posts/raw_csv/"

In [319]:
#This will create a scorecard of each match
def extract_scorecard(file_name):
    #Make csv into dataframe
    balls = pd.read_csv(data + file_name)
    #Get just the runs scored by each batsman and then group them by striker to get runs scored by batsman per match
    runs = balls[['striker','runs_off_bat','innings']].groupby(['striker','innings']).sum().reset_index().rename(columns = {'striker':'batsman'})

    #Did they get out? and if so who and how?
    dismissals = balls[['innings','player_dismissed','bowler','wicket_type']][balls.player_dismissed.notnull()]
    scorecard = pd.merge(runs, dismissals,how='left',left_on=['batsman','innings'], right_on=['player_dismissed','innings'])
    scorecard.drop(columns=['player_dismissed'])

    #How many balls did they face?
    balls_faced = balls[['striker','runs_off_bat','innings']].groupby(['striker','innings']).count().reset_index().rename(columns = {'runs_off_bat':'balls_faced','striker':'batsman'})
    scorecard = pd.merge(scorecard, balls_faced,how='left',on=['batsman','innings'])

    #What position did they bat
    bat_order = pd.melt(balls[['innings', 'ball','striker', 'non_striker']],id_vars=["innings", "ball"]).sort_values(['innings','ball']).reset_index().drop(columns="index")
    #This step is needed for the openers, want the person facing the first ball to be batting #1
    bat_order.loc[bat_order.variable == 'non_striker', 'ball'] = bat_order['ball']+0.01
    #Get lowest ball in an innings that the batsman came in
    bat_order  = bat_order.groupby(['value','innings']).min().sort_values(['innings','ball']).reset_index().rename(columns = {'value':'batsman'})
    #Assign position
    bat_order['position'] = bat_order.groupby('innings').cumcount()+1
    scorecard = pd.merge(scorecard, bat_order,how='left',on=['batsman','innings'])

    #Get the match info and add that in too
    match_info = balls[['match_id','start_date','venue','batting_team','bowling_team']].iloc[0].to_list()
    scorecard['match_id']   = match_info[0]
    scorecard['start_date'] = match_info[1]
    scorecard['match_id'], scorecard['start_date'] , scorecard['venue'] = [match_info[0],match_info[1],match_info[2]]

    #Get who batted in what innings
    innings = balls.loc[balls.ball == 0.1][['innings','batting_team','bowling_team']]
    scorecard = pd.merge(scorecard,innings,how='left',on=['innings'])
    #Get columns in nicer order
    scorecard = scorecard[['match_id','start_date','venue','batting_team','bowling_team','innings','position','batsman', 'runs_off_bat','balls_faced', 'bowler','wicket_type']].rename(columns = {'runs_off_bat':'runs','wicket_type':'how_out'})
    #Sort rows
    scorecard = scorecard.sort_values(['innings','position'])
    return scorecard

In [ ]:
#Get a list of all files where the csvs are stored (only csv files though!)
arr=os.listdir((data))
arr = [x for x in arr if ".csv" in x]
#Then import every one and make it one large dataframe
for file in arr:
    #Might be necessary if one of the imports errors out, uncomment and run again if you need to see which file is causing problems
    #print(file)
    scorecard = extract_scorecard(file)
    if file == arr[0]:
        all_scores = scorecard
    else:
        all_scores = all_scores.append(scorecard)
#Write to csv so we can just import that rather than re-do the whole processing if the notebook goes down
all_scores.to_csv(folder + "all_scores.csv",index=False)

In [4]:
#If necessary, import the csv to get the dataframe
all_scores = pd.read_csv(folder + "all_scores.csv")

In [294]:
#Best to exclude "Not outs", as they're not really scores (switch on or off if change mind)
all_scores_excl = all_scores[pd.notnull(all_scores.how_out)]
def get_freq_count(df_out,df_in,total):
    #Get a count of each run
    df_out = df_in['runs'].value_counts()
    #Make it as a percentage of all innings and then sort
    total = len(df_in)
    print('There are ' + str(total) + ' innings in this dataset.')
    #percentage = (runs/total) *100
    df_out.sort_index()
    return df_out,total
get_freq_count("runs",all_scores_excl,"total")

There are 70097 innings in this dataset.


(0      7928
 1      4071
 2      3085
 4      2868
 5      2514
        ... 
 236       1
 197       1
 245       1
 291       1
 205       1
 Name: runs, Length: 257, dtype: int64,
 70097)

In [233]:
#Plot all scores
df = pd.DataFrame({'Runs':runs.index, 'Count':runs.values})  ## Converting series type to pandas df as plotly accepts dataframe as input. The two columns of df is FuncGroup which is being made by index of series and new variable called count which is made by values of series s.
df['Percentage'] = (df['Count']/total)*100
fig = px.bar(df, x='Runs', y='Percentage')
fig.update_yaxes(range=[0,11.5])
fig.update_xaxes(range=[-0.5,150.5])
fig.show()

In [76]:
#Look at 50 mark
fig = px.bar(df, x='Runs', y='Percentage')
fig.update_yaxes(range=[0,0.7])
fig.update_xaxes(range=[39.5,60.5])

In [77]:
#Look at 100 mark
fig = px.bar(df, x='Runs', y='Percentage')
fig.update_yaxes(range=[0,0.15])
fig.update_xaxes(range=[89.5,110.5])
fig.show()

In [82]:
all_scores.columns

Index(['match_id', 'start_date', 'venue', 'batting_team', 'bowling_team',
       'innings', 'position', 'batsman', 'runs', 'balls_faced', 'bowler',
       'how_out'],
      dtype='object')

In [177]:
#Work out batsmans total runs, times out and times not out
totals = all_scores[['runs','batsman','batting_team']].groupby(['batsman','batting_team']).sum().reset_index()
times_out = all_scores[['how_out','batsman']].groupby(['batsman']).count().reset_index().rename(columns = {'how_out':'n_dismissals'})
all_not_outs = all_scores[pd.isnull(all_scores.how_out)]
all_not_outs = all_not_outs.assign(how_out="not_out")
not_outs =  all_not_outs[['how_out','batsman']].groupby(['batsman']).count().reset_index().rename(columns = {'how_out':'n_not_outs'})

#Now merge all tables as one, make NaNs as 0s
averages = pd.merge(totals,times_out,how="left",on=["batsman"])
averages =pd.merge(averages,not_outs,how="left",on=["batsman"])
averages = averages.fillna(0)

#Calculate average and number of innings
averages['n_inns'] = averages['n_dismissals']+averages['n_not_outs']
averages['average'] = averages['runs']/averages['n_dismissals']
#If average is inf set it to NaN 
averages.loc[np.isinf(averages['average']),'average'] = float('NaN')
     # bat_order.loc[bat_order.variable == 'non_striker', 'ball'] = bat_order['ball']+0.01
all_not_outs = all_scores[pd.isnull(all_scores.how_out)]
#Sort and export as csv
averages = averages.sort_values(['average'],ascending=False)
averages.to_csv(folder + "averages.csv",index=False)
averages

,batsman,batting_team,runs,n_dismissals,n_not_outs,n_inns,average
65,A Rawat,India,153,1,1.0,2.0,153.0
1448,KR Patterson,Australia,144,1,1.0,2.0,144.0
2572,SC Taylor,England,250,2,3.0,5.0,125.0
918,GK Diviya,Singapore,111,1,2.0,3.0,111.0
215,Aamer Yamin,Pakistan,110,1,3.0,4.0,110.0
...,...,...,...,...,...,...,...
3151,ZK Finkili,Tanzania,8,0,1.0,1.0,NaN
3152,ZK Shem,Vanuatu,1,0,1.0,1.0,NaN
3158,Zahid Shah,United Arab Emirates,11,0,2.0,2.0,NaN
3163,Zakiullah Zaki,Afghanistan,3,0,1.0,1.0,NaN


In [228]:
#Merge with all scores
combined_scores = pd.merge(all_scores_excl,averages.rename(columns = {'runs':'total_runs'}),how='left',on=['batsman','batting_team'])
#Identify "good" batsmen, assume bad by default
combined_scores['quality'] = 'Bad'
combined_scores.loc[(combined_scores['average']>=25) & (combined_scores['total_runs']>= 1000),['quality']] = 'Good'

In [298]:
#subset good and bad scores and make counts
good_scores = combined_scores[combined_scores['quality']=='Good']
bad_scores = combined_scores[combined_scores['quality']=='Bad']

good_runs,good_total = get_freq_count("good_runs",good_scores,"good_total")
bad_runs,bad_total = get_freq_count("bad_runs",bad_scores,"bad_total")

#Plot all scores
def change_to_df(df_in,df_out,quality,total):
    df_out = pd.DataFrame({'Runs':df_in.index, 'Count':df_in.values})  ## Converting series type to pandas df as plotly accepts dataframe as input. The two columns of df is FuncGroup which is being made by index of series and new variable called count which is made by values of series s.
    df_out['Percentage'] = (df_out['Count']/total)*100
    df_out['quality'] = quality
    df_out = df_out.sort_values(['Runs'])
    return df_out
 
good_df = change_to_df(good_runs,"good_df","Good",good_total)
bad_df = change_to_df(bad_runs,"bad_df","Bad",bad_total)
both = good_df.append(bad_df)
fig = px.line(both, x='Runs', y='Percentage',color='quality')
#fig.update_yaxes(range=[0,11.5])
fig.update_xaxes(range=[-0.5,50.5])
fig.show()

There are 37160 innings in this dataset.
There are 32937 innings in this dataset.


In [ ]:
#Plot distribution of only "good" batsmen